# Guided Lab — Chapter 6  
## Integrating Domain Objects with Python’s Data Model
  
**Chapter:** 6 — Special Methods, Operators, and Contracts    

---

## Learning Goals

By the end of this lab, you will be able to:

- Implement special methods to integrate custom objects into Python’s data model
- Design effective `__repr__` and `__str__` methods for different audiences
- Implement correct equality, hashing, and ordering semantics
- Apply operator overloading responsibly and defensively
- Use Abstract Base Classes to define behavioral contracts
- Explain and justify immutability decisions in domain objects

---
## Setup

In [ ]:
from dataclasses import dataclass
from decimal import Decimal
from abc import ABC, abstractmethod
from functools import total_ordering

---
## Part 1 — A Domain Problem That Looks Simple (But Isn’t)

We are going to design a `Money` object that represents a monetary value.

At first glance, this seems trivial — just a number and a currency.

In practice, monetary values expose many of the hardest design problems in Python:

- equality vs identity  
- mutability vs hashability  
- operator overloading  
- representation for humans vs developers  

Our goal is not to build a financial system, but to build **one correct, Pythonic value object**.

---
## Part 2 — Declaring a Behavioral Contract with an ABC
### Step 1: Define the Contract

In [ ]:
class MonetaryValue(ABC):
    """
    Abstract base class defining the behavior expected
    of monetary-like value objects.
    """

    @abstractmethod
    def __add__(self, other):
        pass

    @abstractmethod
    def __sub__(self, other):
        pass

    @abstractmethod
    def __eq__(self, other) -> bool:
        pass

    def is_zero(self) -> bool:
        """
        Default implementation.
        Subclasses may override if necessary.
        """
        return False

### Guided Questions

- Why does this ABC focus on **behavior** rather than attributes?
- Why are operator methods appropriate candidates for abstraction?
- What kind of errors does this prevent later?

---
## Part 3 — Creating an Immutable Value Object
### Step 2: Define the `Money` Class Skeleton

We choose immutability **before** writing any methods. This decision affects hashing, equality, and operator behavior.

In [ ]:
@total_ordering
@dataclass(frozen=True)
class Money(MonetaryValue):
    amount: Decimal
    currency: str

    # We will implement special methods below.
    pass

---
## Part 4 — Representation: Debugging vs Display
### Step 3–4: Implement `__repr__` and `__str__`

We will redefine `Money` with the special methods included.

In [ ]:
@total_ordering
@dataclass(frozen=True)
class Money(MonetaryValue):
    amount: Decimal
    currency: str

    def __repr__(self) -> str:
        # Developer-facing: unambiguous and information-dense
        return f"Money(amount={self.amount!r}, currency={self.currency!r})"

    def __str__(self) -> str:
        # User-facing: readable summary
        return f"{self.currency} {self.amount}"

### Guided Exercise

Run this cell and observe how `print()` differs from the notebook display output.

In [ ]:
m = Money(Decimal("19.99"), "USD")
print(m)
m

---
## Part 5 — Equality Semantics
### Step 5: Implement `__eq__`

We will redefine `Money` again, now including equality. Note the use of `NotImplemented`.

In [ ]:
@total_ordering
@dataclass(frozen=True)
class Money(MonetaryValue):
    amount: Decimal
    currency: str

    def __repr__(self) -> str:
        return f"Money(amount={self.amount!r}, currency={self.currency!r})"

    def __str__(self) -> str:
        return f"{self.currency} {self.amount}"

    def __eq__(self, other) -> bool:
        if not isinstance(other, Money):
            return NotImplemented
        return self.amount == other.amount and self.currency == other.currency

### Guided Questions

- Why return `NotImplemented` instead of `False`?
- What happens if we compare `Money` to an `int`?
- What does equality *mean* in this domain?

In [ ]:
Money(Decimal("10.00"), "USD") == Money(Decimal("10.00"), "USD")

In [ ]:
Money(Decimal("10.00"), "USD") == 10

---
## Part 6 — Hashing and the Equality Contract
### Step 6: Implement `__hash__`

Because `Money` is immutable, it is safe to be hashable. We'll define hashing based on the same fields used for equality.

In [ ]:
@total_ordering
@dataclass(frozen=True)
class Money(MonetaryValue):
    amount: Decimal
    currency: str

    def __repr__(self) -> str:
        return f"Money(amount={self.amount!r}, currency={self.currency!r})"

    def __str__(self) -> str:
        return f"{self.currency} {self.amount}"

    def __eq__(self, other) -> bool:
        if not isinstance(other, Money):
            return NotImplemented
        return self.amount == other.amount and self.currency == other.currency

    def __hash__(self) -> int:
        return hash((self.amount, self.currency))

### Guided Experiment

Verify that equal objects have equal hashes, and that dictionary lookup works as expected.

In [ ]:
a = Money(Decimal("10.00"), "USD")
b = Money(Decimal("10.00"), "USD")

print(a == b)
print(hash(a), hash(b))

wallet = {a: "stored"}
print(wallet[b])

---
## Part 7 — Ordering with Constraints
### Step 7: Implement ordering (`__lt__`)

We'll define ordering only when currencies match. Otherwise we raise a meaningful error.

In [ ]:
@total_ordering
@dataclass(frozen=True)
class Money(MonetaryValue):
    amount: Decimal
    currency: str

    def __repr__(self) -> str:
        return f"Money(amount={self.amount!r}, currency={self.currency!r})"

    def __str__(self) -> str:
        return f"{self.currency} {self.amount}"

    def __eq__(self, other) -> bool:
        if not isinstance(other, Money):
            return NotImplemented
        return self.amount == other.amount and self.currency == other.currency

    def __hash__(self) -> int:
        return hash((self.amount, self.currency))

    def __lt__(self, other):
        if not isinstance(other, Money):
            return NotImplemented
        if self.currency != other.currency:
            raise ValueError("Cannot compare Money values with different currencies")
        return self.amount < other.amount

### Guided Questions

- Why is ordering more dangerous than equality?
- Why do we raise an exception instead of silently comparing?
- When would *not* implementing ordering be the right choice?

In [ ]:
Money(Decimal("5.00"), "USD") < Money(Decimal("7.00"), "USD")

In [ ]:
try:
    Money(Decimal("5.00"), "USD") < Money(Decimal("7.00"), "EUR")
except ValueError as e:
    print("Expected error:", e)

---
## Part 8 — Operator Overloading
### Step 8–9: Implement `+` and `-`

We will allow addition/subtraction only for matching currencies, return new objects, and return `NotImplemented` for unsupported types.

In [ ]:
@total_ordering
@dataclass(frozen=True)
class Money(MonetaryValue):
    amount: Decimal
    currency: str

    def __repr__(self) -> str:
        return f"Money(amount={self.amount!r}, currency={self.currency!r})"

    def __str__(self) -> str:
        return f"{self.currency} {self.amount}"

    def __eq__(self, other) -> bool:
        if not isinstance(other, Money):
            return NotImplemented
        return self.amount == other.amount and self.currency == other.currency

    def __hash__(self) -> int:
        return hash((self.amount, self.currency))

    def __lt__(self, other):
        if not isinstance(other, Money):
            return NotImplemented
        if self.currency != other.currency:
            raise ValueError("Cannot compare Money values with different currencies")
        return self.amount < other.amount

    def __add__(self, other):
        if not isinstance(other, Money):
            return NotImplemented
        if self.currency != other.currency:
            raise ValueError("Cannot add Money values with different currencies")
        return Money(self.amount + other.amount, self.currency)

    def __sub__(self, other):
        if not isinstance(other, Money):
            return NotImplemented
        if self.currency != other.currency:
            raise ValueError("Cannot subtract Money values with different currencies")
        return Money(self.amount - other.amount, self.currency)

    def is_zero(self) -> bool:
        return self.amount == Decimal("0")

### Guided Exercise

In [ ]:
m1 = Money(Decimal("50.00"), "USD")
m2 = Money(Decimal("20.00"), "USD")

print("m1 + m2 =", m1 + m2)
print("m1 - m2 =", m1 - m2)

In [ ]:
try:
    Money(Decimal("1.00"), "USD") + Money(Decimal("1.00"), "EUR")
except ValueError as e:
    print("Expected error:", e)

---
## Part 9 — Bonus Operator: Scalar Multiplication
### Step 10: Implement `Money * scalar`

We will support multiplying by an `int` or `Decimal`. Multiplying `Money * Money` is not supported.

In [ ]:
@total_ordering
@dataclass(frozen=True)
class Money(MonetaryValue):
    amount: Decimal
    currency: str

    def __repr__(self) -> str:
        return f"Money(amount={self.amount!r}, currency={self.currency!r})"

    def __str__(self) -> str:
        return f"{self.currency} {self.amount}"

    def __eq__(self, other) -> bool:
        if not isinstance(other, Money):
            return NotImplemented
        return self.amount == other.amount and self.currency == other.currency

    def __hash__(self) -> int:
        return hash((self.amount, self.currency))

    def __lt__(self, other):
        if not isinstance(other, Money):
            return NotImplemented
        if self.currency != other.currency:
            raise ValueError("Cannot compare Money values with different currencies")
        return self.amount < other.amount

    def __add__(self, other):
        if not isinstance(other, Money):
            return NotImplemented
        if self.currency != other.currency:
            raise ValueError("Cannot add Money values with different currencies")
        return Money(self.amount + other.amount, self.currency)

    def __sub__(self, other):
        if not isinstance(other, Money):
            return NotImplemented
        if self.currency != other.currency:
            raise ValueError("Cannot subtract Money values with different currencies")
        return Money(self.amount - other.amount, self.currency)

    def __mul__(self, factor):
        if not isinstance(factor, (int, Decimal)):
            return NotImplemented
        return Money(self.amount * Decimal(factor), self.currency)

    def is_zero(self) -> bool:
        return self.amount == Decimal("0")

### Guided Exercise

In [ ]:
price = Money(Decimal("9.99"), "USD")
print("price * 3 =", price * 3)
print("price * Decimal('2.5') =", price * Decimal("2.5"))

---
## Part 10 — Pulling It All Together
### Final Integrated Checks

In [ ]:
prices = {
    Money(Decimal("9.99"), "USD"),
    Money(Decimal("19.99"), "USD"),
    Money(Decimal("9.99"), "USD")  # duplicate by value
}

print("Set size should be 2:", len(prices))
prices

### Reflection

In a few sentences, answer the following:

- What design choice made set membership safe?
- What would break if `Money` were mutable?
- Which special method in this lab has the largest “blast radius” if implemented incorrectly?